# Домашнее задание 1

### Описание

В вашем распоряжении датасет с русскоязычными отзывами о мобильных телефонах с выставленным рейтингом от 1 до 5.
Ключевая задача – обучить любую модель регрессии (или классификации, если решите таким путём пойти) из пакетов scikit, XGBoost, LightGBM, CatBoost.


Необходимая метрика:

1. Со звёздочкой (дополнительный балл) – MAE <= 0.5
2. Минимальное допустимое значение – МАЕ <= 1.0

### Что необходимо сделать

1. Откройте датасет
2. Разделите на обучение и тест
3. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 
4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).
5. Вывести ТОП-50 наиболее частотных токенов:
- только для униграмм
- только для биграмм
- только для триграмм
- для всех вариантов n-грамм одновременно

Напишите, какие наблюдения и выводы есть.
6. Повторите пункт 5, только отдельно для отзывов с рейтингом «4-5», «3» и «1-2». Есть ли ключевые отличия? Есть кандидаты на попадание в список стоп-слов?

7. Составьте список своих ключевых слов, для помощи можно использовать nltk и punctuations из string. 
8. Закодируйте полученные отзывы с помощью CountVectorizer и TfIdfVectorizer (экспериментируйте с параметрами min_df  и max_df). 
9. Обучение одну или несколько моделей машинного обучения на разных представлениях данных
10. Валидируйте модель. Если модель соответствует условиям метрик, то работа завершена. В ином случае, экспериментируйте, начиная с пункта 7. 
11. По всем попыткам обучить качественную модель пишите свои выводы и замечания, почему так получилось.


## Комментарии исполнителя

Я изменил порядок 2 и 3 пункта, потому что так намного удобнее

## 0. Импорт библиотк, определение констант

In [3]:
import os
import pandas as pd
import numpy as np
import spacy
import re

from string import punctuation
from collections import Counter

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


from pymorphy3 import MorphAnalyzer

from nltk import ngrams
from nltk.corpus import stopwords

from sklearn.linear_model import LinearRegression

from catboost import CatBoost, CatBoostRegressor

from sklearn.metrics import classification_report

In [3]:
list(punctuation)

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [4]:
tqdm.pandas()

In [5]:
#!python -m spacy download ru_core_news_sm

## 1. Откройте датасет

In [4]:
if os.path.exists("data/data_lemma_cleared.csv"):
    df = pd.read_csv("data/data_lemma_cleared.csv", engine='python')
elif os.path.exists("data/data_lemma.csv"):
    df = pd.read_csv("data/data_lemma.csv", engine='python')
else:    
    df = pd.read_csv("data/data.csv")

In [5]:
df.shape

(322090, 4)

In [6]:
df.head(10)

,Unnamed: 0,Review,Rating,lemma
0,0,3d touch просто восхитительная вещь! заряд дер...,5,3d touch просто восхитительный вещь! заряд дер...
1,1,"отключается при температуре близкой к нулю, не...",4,"отключаться температура близкий нулю, непонятн..."
2,2,"в apple окончательно решили не заморачиваться,...",3,"apple окончательно решить не заморачиваться, д..."
3,3,постарался наиболее ёмко и коротко описать все...,4,постараться наиболее ёмко коротко описать всё ...
4,4,достойный телефон. пользоваться одно удовольст...,5,достойный телефон. пользоваться удовольствие.
5,5,6s gold 64gb,5,6s gold 64gb
6,6,мой первый айфон. скажу честно- эппл ранее ник...,5,первый айфон. сказать честно- эппл ранее рука ...
7,7,мне очень понравилась эта модель! во-первых кл...,5,очень понравиться модель! во-первых классный д...
8,8,долгое время пользовалась iphone 5s 16gb. он м...,5,долгий время пользоваться iphone 5s 16gb. устр...
9,9,"раньше был samsung galaxy alpha sm-g850f 32gb,...",4,"ранний samsung galaxy alpha sm-g850f 32gb, куп..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322090 entries, 0 to 322089
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  322090 non-null  object
 1   Review      321500 non-null  object
 2   Rating      320556 non-null  object
 3   lemma       319591 non-null  object
dtypes: object(4)
memory usage: 9.8+ MB


In [8]:
df = df.dropna().reset_index(drop=True)
df.head()

,Unnamed: 0,Review,Rating,lemma
0,0,3d touch просто восхитительная вещь! заряд дер...,5,3d touch просто восхитительный вещь! заряд дер...
1,1,"отключается при температуре близкой к нулю, не...",4,"отключаться температура близкий нулю, непонятн..."
2,2,"в apple окончательно решили не заморачиваться,...",3,"apple окончательно решить не заморачиваться, д..."
3,3,постарался наиболее ёмко и коротко описать все...,4,постараться наиболее ёмко коротко описать всё ...
4,4,достойный телефон. пользоваться одно удовольст...,5,достойный телефон. пользоваться удовольствие.


In [9]:
df['Rating'] = df['Rating'].astype('Int8')
df['Rating'].value_counts()

Rating
5    159355
4     74969
3     37067
2     24580
1     23620
Name: count, dtype: Int64

In [10]:
df = df[(df['Rating'] >= 1) & (df['Rating'] <= 5)]

In [11]:
df['Rating'].value_counts(normalize=True)

Rating
5    0.498622
4    0.234578
3    0.115983
2    0.076911
1    0.073907
Name: proportion, dtype: Float64

Я закомментил все что ниже, потому что такую обработку лучше делать после лемматизации

In [12]:
df.duplicated().sum()

0

In [13]:
df[df.duplicated()].head(10)

,Unnamed: 0,Review,Rating,lemma


In [14]:
df["Review"].value_counts()

Review
не советую.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Основная причина дублей - короткие одинаковые комментарии, такие тубли можно исключить

In [15]:
# df = df.drop_duplicates()

In [16]:
df["Review"].value_counts()

Review
не советую.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [17]:
df[df["Review"] == "-"].value_counts()

Unnamed: 0  Review  Rating  lemma
17023       -       3       -        1
20072       -       5       -        1
31346       -       2       -        1
451846      -       1       -        1
98141       -       4       -        1
Name: count, dtype: int64

Исключим записи с одинаковым комментарием, но выоским разрбросом оценки

In [18]:
#bad_comments = df["Review"].value_counts()
#bad_comments = bad_comments[bad_comments >= 3]
#bad_comments.head(20)

#bad_comments = bad_comments[bad_comments['value'].isin((4,5))]

In [19]:
df['Review'] = df['Review'].str.lower()

Уберём синтаксические знаки

In [20]:
punct = list(punctuation)
punct.remove(":")
punct.remove(")")
punct.remove("(")
punct = ''.join(punct)

In [21]:
df['Review']

0         3d touch просто восхитительная вещь! заряд дер...
1         отключается при температуре близкой к нулю, не...
2         в apple окончательно решили не заморачиваться,...
3         постарался наиболее ёмко и коротко описать все...
4         достойный телефон. пользоваться одно удовольст...
                                ...                        
319586    удобный, всё работает отлично, звонит, играет,...
319587    прошло больше года, притензий нет, при моей на...
319588    мой первый аппарат на андроиде. на данный моме...
319589    разбил iphone и не было желания покупать новый...
319590             очень доволен покупкой и всем советую...
Name: Review, Length: 319591, dtype: object

In [22]:
punct

'!"#$%&\'*+,-./;<=>?@[\\]^_`{|}~'

In [23]:
df['Review'] = df['Review'].str.replace(r'[' + punct + ']', '', regex=True)

## 2. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 

In [24]:
stop_words = set(stopwords.words('russian'))
stop_words.remove("не"); 

### spacy

In [25]:
spacy_nlp = spacy.load("ru_core_news_sm")

In [26]:
def spacy_lemmatize_text(text):
    try:
        doc = spacy_nlp(text)
    except:
        print(f'{text}')
        return ""
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

В этом месте у меня падала лемматизация, так я обнаружил что есть пустые комменты, но трай эксепт оставил, в пайморфек уже нет этого, потому что мусор удален

### pymorphy3

In [27]:
pymorphy3_analyzer = MorphAnalyzer()

In [28]:
def pymorphy3_lemmatize_text(text):
    lemmas = [pymorphy3_analyzer.parse(word)[0].normal_form for word in text.split()]
    clean_tokens = [w for w in lemmas if not w in stop_words]  
    return (' '.join(clean_tokens))

### Лемматинизируем

In [29]:
if not "lemma" in df.columns:
    print('Лемматизируем текст');
    df["lemma"] = df["Review"].progress_apply(lambda row: pymorphy3_lemmatize_text(row))
    df.to_csv("data/data_lemma_.csv", index=False)

In [30]:
df.shape

(319591, 4)

In [31]:
df = df.dropna()

In [32]:
df = df.drop_duplicates()

In [33]:
df.shape

(319591, 4)

In [34]:
if not os.path.exists("data/data_lemma_cleared.csv"):
    df.to_csv("data/data_lemma_cleared.csv")

In [35]:
df

,Unnamed: 0,Review,Rating,lemma
0,0,3d touch просто восхитительная вещь заряд держ...,5,3d touch просто восхитительный вещь! заряд дер...
1,1,отключается при температуре близкой к нулю неп...,4,"отключаться температура близкий нулю, непонятн..."
2,2,в apple окончательно решили не заморачиваться ...,3,"apple окончательно решить не заморачиваться, д..."
3,3,постарался наиболее ёмко и коротко описать все...,4,постараться наиболее ёмко коротко описать всё ...
4,4,достойный телефон пользоваться одно удовольствие,5,достойный телефон. пользоваться удовольствие.
...,...,...,...,...
319586,457832,удобный всё работает отлично звонит играет сни...,5,"удобный, всё работать отлично, звонит, играет,..."
319587,457833,прошло больше года притензий нет при моей нагр...,5,"пройти большой года, притензия нет, нагрузка 5..."
319588,457834,мой первый аппарат на андроиде на данный момен...,5,первый аппарат андроиде. данный момент не раза...
319589,457835,разбил iphone и не было желания покупать новый...,5,"разбить iphone не желание покупать новый, хоте..."


## 3. Разделите на обучение и тест

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df['lemma'], df['Rating'], test_size=0.2, stratify=df['Rating'])

print(X_train.shape)
print(y_train.shape)

print(X_test.shape)
print(y_test.shape)

(255672,)
(255672,)
(63919,)
(63919,)


## 4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).

Я всё таки попробую разные способы, чтобы сравнить быстродействие и результат

In [39]:
def ngrams_stas(data, n):
    word_vectorizer = CountVectorizer(ngram_range=(n,n), analyzer='word')
    sparse_matrix = word_vectorizer.fit_transform(data)
    frequencies = sum(sparse_matrix).data
    return pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names_out(), columns=['frequency'])

### Униграммы

In [40]:
unigrams = list(df['lemma'].str.split()) 

In [41]:
unigrams = [x for xs in unigrams for x in xs]

In [42]:
len(unigrams)

14821509

In [43]:
print(len(set(unigrams)))

685442


In [44]:
unigrams

['3d',
 'touch',
 'просто',
 'восхитительный',
 'вещь!',
 'заряд',
 'держать',
 'целый',
 'день.',
 'розовый',
 'цвет',
 'смотреться',
 'очень',
 'необычно.',
 'touch',
 'id',
 'очень',
 'быстрый',
 'удобный.',
 'весь',
 'советовать',
 'телефон!',
 'отключаться',
 'температура',
 'близкий',
 'нулю,',
 'непонятно',
 'вести',
 'батарея',
 'apple',
 'окончательно',
 'решить',
 'не',
 'заморачиваться,',
 'делать',
 'незначительный',
 'изменение',
 'телефоне,',
 'выдавать',
 'изменение',
 'инновации.',
 'скопировать',
 'не',
 'функционал,',
 'дизайн.',
 'цена',
 'неадекватно',
 'завышена,',
 'скачок',
 'курс',
 'говорить',
 'это',
 'не',
 'приходится.',
 'многие',
 'скажут,',
 'hd',
 'разрешение',
 'достаточно',
 'диагонали,',
 'мочь',
 'оно',
 'есть,',
 'аппарат',
 'позиционируется,',
 'топовый',
 'решение,',
 'итог',
 'получать',
 'банальный',
 'экономия',
 'производителя,',
 'произойти',
 'время',
 'работы,',
 'увеличить',
 'fhd?!',
 '1',
 'гб',
 'оперативка',
 'прошлый',
 'век!',
 'не',

In [45]:
unigrams_counter = Counter(unigrams)
unigrams_counter.most_common(50)

[('не', 599554),
 ('телефон', 257410),
 ('-', 217833),
 ('это', 196225),
 ('всё', 144682),
 ('очень', 136518),
 ('хороший', 90544),
 ('купить', 65713),
 ('весь', 61574),
 ('свой', 59487),
 ('экран', 59337),
 ('аппарат', 54317),
 ('работать', 54135),
 ('который', 53936),
 ('просто', 53847),
 ('камера', 52740),
 ('пользоваться', 50746),
 ('мочь', 49872),
 ('ещё', 49321),
 ('день', 45541),
 ('год', 44428),
 ('большой', 43480),
 ('2', 38890),
 ('брать', 38568),
 ('цена', 37763),
 ('время', 35584),
 ('смартфон', 34537),
 ('качество', 34349),
 ('покупать', 32931),
 ('батарея', 32820),
 ('первый', 32792),
 ('отличный', 31706),
 ('проблема', 30813),
 ('модель', 30771),
 ('приложение', 29233),
 ('пока', 27540),
 ('стать', 27461),
 ('деньга', 27034),
 ('покупка', 26821),
 ('использование', 26462),
 ('держать', 26081),
 ('самый', 25584),
 ('работа', 25482),
 ('хотеть', 25468),
 ('целое', 24755),
 ('месяц', 24415),
 ('стоить', 24286),
 ('новый', 24204),
 ('нужный', 24158),
 ('телефон,', 24149)]

Попробуем создать нграммы через универсальную функцию с помощью CountVectorizer

In [46]:
#ngrams_stas(df['lemma'], 1).sort_values('frequency', ascending=False)

Получился странный и непонятный мне результат

### биграммы и триграммы
попробуем через nltk

In [47]:

un, bi, th = [], [], []
for s in df['lemma']:
    sent = s.split()
    un.append(list(ngrams(sent, 1)))
    bi.append(list(ngrams(sent, 2)))
    th.append(list(ngrams(sent, 3)))

Работаем мнгновенно!

In [48]:
un = [x for xs in un for x in xs]
un_counter = Counter(un)
un_counter.most_common(50)

[(('не',), 599554),
 (('телефон',), 257410),
 (('-',), 217833),
 (('это',), 196225),
 (('всё',), 144682),
 (('очень',), 136518),
 (('хороший',), 90544),
 (('купить',), 65713),
 (('весь',), 61574),
 (('свой',), 59487),
 (('экран',), 59337),
 (('аппарат',), 54317),
 (('работать',), 54135),
 (('который',), 53936),
 (('просто',), 53847),
 (('камера',), 52740),
 (('пользоваться',), 50746),
 (('мочь',), 49872),
 (('ещё',), 49321),
 (('день',), 45541),
 (('год',), 44428),
 (('большой',), 43480),
 (('2',), 38890),
 (('брать',), 38568),
 (('цена',), 37763),
 (('время',), 35584),
 (('смартфон',), 34537),
 (('качество',), 34349),
 (('покупать',), 32931),
 (('батарея',), 32820),
 (('первый',), 32792),
 (('отличный',), 31706),
 (('проблема',), 30813),
 (('модель',), 30771),
 (('приложение',), 29233),
 (('пока',), 27540),
 (('стать',), 27461),
 (('деньга',), 27034),
 (('покупка',), 26821),
 (('использование',), 26462),
 (('держать',), 26081),
 (('самый',), 25584),
 (('работа',), 25482),
 (('хотеть',

Совпадает с тем, что делал вручную и работает достаточно быстро. Делаем универсальную функцию

In [49]:
def ngrams_stats_nltk(data):
    
    un, bi, th = [], [], []
    
    for s in data:
        sent = s.split()
        un.append(list(ngrams(sent, 1)))
        bi.append(list(ngrams(sent, 2)))
        th.append(list(ngrams(sent, 3)))
        
    un = [x for xs in un for x in xs]
    un = Counter(un)
    
    bi = [x for xs in bi for x in xs]
    bi = Counter(bi)
    
    th = [x for xs in th for x in xs]
    th = Counter(th)       
    
    return pd.DataFrame({
                        'Униграмы' : un.most_common(50),
                        'Биграммы' : bi.most_common(50),
                        'Триграммы' : th.most_common(50)  
                        })
      
        

## 5. Топы

### Топ 50 для всех отзывов

In [49]:
ngrams_stats_nltk(df["lemma"])


,Униграмы,Биграммы,Триграммы
0,"((не,), 599554)","((это, не), 17716)","((стоить, свой, денег.), 1270)"
1,"((телефон,), 257410)","((телефон, не), 16743)","((игра, не, играю,), 1007)"
2,"((-,), 217833)","((-, это), 13374)","((сей, пора, не), 1002)"
3,"((это,), 196225)","((-, не), 12046)","((-, это, не), 932)"
4,"((всё,), 144682)","((не, мочь), 11386)","((стоить, свой, деньга), 777)"
5,"((очень,), 136518)","((пока, не), 9148)","((целое, телефон, очень), 771)"
6,"((хороший,), 90544)","((телефон, очень), 8719)","((не, советовать, покупать), 727)"
7,"((купить,), 65713)","((это, телефон), 8495)","((стоить, свой, денег,), 726)"
8,"((весь,), 61574)","((не, очень), 7780)","((отличный, телефон, свой), 716)"
9,"((свой,), 59487)","((свой, деньга), 7366)","((всё, равно, не), 699)"


## 6. Топы по оценкам

### Топ 50 для плохих отзывов

In [50]:
ngrams_stats_nltk(df.loc[df["Rating"] < 3, "lemma"])

,Униграмы,Биграммы,Триграммы
0,"((не,), 100129)","((телефон, не), 3503)","((не, советовать, покупать), 505)"
1,"((телефон,), 41437)","((это, не), 2897)","((не, советовать, брать), 394)"
2,"((это,), 31922)","((не, советовать), 2658)","((не, стоить, свой), 350)"
3,"((-,), 30251)","((не, мочь), 2258)","((никто, не, советовать), 349)"
4,"((всё,), 17342)","((-, не), 1973)","((большой, не, купить), 320)"
5,"((очень,), 13316)","((-, это), 1945)","((кой, случай, не), 275)"
6,"((купить,), 12755)","((большой, не), 1772)","((свой, деньга, не), 168)"
7,"((просто,), 9289)","((не, покупать), 1603)","((не, покупать, телефон), 160)"
8,"((хороший,), 9032)","((никто, не), 1557)","((телефон, не, стоить), 158)"
9,"((мочь,), 8410)","((это, телефон), 1500)","((стоить, свой, денег.), 157)"


In [51]:
ngrams_stats_nltk(df.loc[df["Rating"] == 3, "lemma"])

,Униграмы,Биграммы,Триграммы
0,"((не,), 74555)","((телефон, не), 2557)","((не, стоить, свой), 167)"
1,"((телефон,), 31630)","((это, не), 2175)","((не, советовать, покупать), 138)"
2,"((-,), 25856)","((не, мочь), 1534)","((свой, деньга, не), 136)"
3,"((это,), 23411)","((-, не), 1515)","((телефон, не, плохой,), 120)"
4,"((всё,), 14882)","((-, это), 1507)","((не, советовать, брать), 116)"
5,"((очень,), 12006)","((не, очень), 1115)","((стоить, свой, денег.), 112)"
6,"((хороший,), 9089)","((это, телефон), 1068)","((всё, равно, не), 109)"
7,"((купить,), 7815)","((пока, не), 1057)","((оставлять, желать, лучшего.), 102)"
8,"((экран,), 7165)","((большой, не), 821)","((-, это, не), 98)"
9,"((мочь,), 6829)","((всё, равно), 799)","((целое, телефон, не), 94)"


### Топ 50 для хороших отзывов

In [52]:
ngrams_stats_nltk(df.loc[df["Rating"] > 3, "lemma"])

,Униграмы,Биграммы,Триграммы
0,"((не,), 424870)","((это, не), 12644)","((стоить, свой, денег.), 1001)"
1,"((телефон,), 184343)","((телефон, не), 10683)","((игра, не, играю,), 866)"
2,"((-,), 161726)","((-, это), 9922)","((сей, пора, не), 812)"
3,"((это,), 140892)","((-, не), 8558)","((целое, телефон, очень), 745)"
4,"((всё,), 112458)","((телефон, очень), 7612)","((отличный, телефон, свой), 696)"
5,"((очень,), 111196)","((не, мочь), 7594)","((-, это, не), 681)"
6,"((хороший,), 72423)","((пока, не), 7022)","((стоить, свой, деньга), 631)"
7,"((весь,), 47696)","((свой, деньга), 6150)","((телефон, стоить, свой), 610)"
8,"((свой,), 46673)","((не, очень), 5965)","((удобно, лежать, руке,), 597)"
9,"((купить,), 45143)","((это, телефон), 5927)","((сканер, отпечаток, палец), 565)"


Смотреть объединенную статистику бессмысленно, так как частотность униграм превышает частотность би и триграмм, соотвественном будет состоять из униграм.

### Вывод

Чем выше размер энграм , тем существенней отличия в положительных и отрицательных отзывах. По триграммам отлично понятно - положительный или отрицательный отзыв. Гипотеза - фичи на триграммах дадут лучший результат. 
Заметил присутствие слова "это", думаю никакой смысловой нагрузке в данном случае оно не несёт и его можно смело удалить. Так как и слова "телефон" , "сей", "пора", "смартфон" . Остальное оставлю, чтобы не портить значащие би и три граммы.

## 7. Список стоп слов

Базовый список стоп слов и пунктуацию мы использовали в предыдущиз шагах, в этом сделаем дополнительный список на основании анализа энграм

In [37]:
custom_stopwords = ['это', 'телефон', 'сей', 'пора', 'смартфон']

## 8. Закодируйте полученные отзывы с помощью CountVectorizer и TfIdfVectorizer 

(экспериментируйте с параметрами min_df  и max_df). 

В интернетах пишут
So to sum it up, pruning the terms via min_df and max_df is to improve the performance, not the quality of clusters (as an example).
а так как перфоманс меня вполне себе устраивает, то я с этими параметрами экспериментирова не буду, качество гарантированно не улучшится

### CountVectorizer

In [54]:
vec = CountVectorizer(ngram_range=(3, 3))

X_train_embeddings = vec.fit_transform(X_train)

In [55]:
clf = LinearRegression()
clf.fit(X_train_embeddings, y_train)

y_pred = clf.predict(vec.transform(X_test))
y_pred = np.round(y_pred)

print(mean_absolute_error(y_test, y_pred))

0.5655908258890158


### Биграммы

In [56]:
n_grams = (2, 2)

vec = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords)
X_train_embeddings = vec.fit_transform(X_train)

lr = LinearRegression()
lr.fit(X_train_embeddings, y_train)

y_pred = lr.predict(vec.transform(X_test))
y_pred = np.round(y_pred)
    
print(mean_absolute_error(y_test, y_pred))

0.4817972746757615


### Триграммы

In [57]:
n_grams = (3, 3)

vec = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords)
X_train_embeddings = vec.fit_transform(X_train)

lr = LinearRegression()
lr.fit(X_train_embeddings, y_train)

y_pred = lr.predict(vec.transform(X_test))
y_pred = np.round(y_pred)
    
print(mean_absolute_error(y_test, y_pred))

0.4865689388131854


### би+три граммы

In [ ]:
n_grams = (2, 3)

tfidf_vect = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords)
X_train_embeddings = tfidf_vect.fit_transform(X_train)

lr = LinearRegression()
lr.fit(X_train_embeddings, y_train)

y_pred = lr.predict(tfidf_vect.transform(X_test))
y_pred = np.round(y_pred)
    
print(mean_absolute_error(y_test, y_pred))

0.4285267291415698


### CatBoost

Кэтбуст со встроенным векторайзером

In [59]:
text_features = ['lemma']

In [60]:
X_train_catboost = pd.DataFrame({'lemma': X_train})

In [61]:
model_cb = CatBoostRegressor(learning_rate=0.005,
                            max_depth=10,
                            n_estimators=5000,
                            loss_function='MAE',
                            thread_count=-1, 
                            random_seed=42, 
                            verbose=0, 
                            text_features=text_features,
                            task_type = "GPU" 
                                
)


model_cb.fit(X_train_catboost, y_train, text_features=text_features, plot=True)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because MAE is/are not implemented for GPU


In [62]:
X_test_catboost = pd.DataFrame({'lemma': X_test})
 
y_pred = model_cb.predict(X_test_catboost)
y_pred = np.round(y_pred)

print(mean_absolute_error(y_test, y_pred))


0.799261565418733


Кэтбус на фичах TF-IDF

In [40]:
X_train

16187     общем, полно положительный эмосиус. покупать н...
171011    телефон говно, брать девушке, создать лишний г...
223119    смартфон купить подарок брату, аппарат достойн...
143167    это пользоваться гэлэксить ноут 3, поэтому сра...
146945    покупать телефон не советовать никому, покупат...
                                ...                        
193528    долго выбирать замена свой siemens sx1 купить ...
303055    батарея больший ёмкость народный смартфон мнен...
214638    это нереально хороший телефон!!! тех, любить п...
280375    жить год (10 месяцев). правда уронить асфальт ...
220220    нормальный смартфон казаться nokia доба сменит...
Name: lemma, Length: 255672, dtype: object

In [38]:
n_grams = (2, 3)

vec = TfidfVectorizer(ngram_range=n_grams, stop_words=custom_stopwords, min_df=20, max_df=0.95)
X_train_embeddings = vec.fit_transform(X_train)
X_train_embeddings.shape

(255672, 64051)

In [ ]:
%%time

model_cb = CatBoostRegressor(thread_count=-1,
                             loss_function='MAE',
                             early_stopping_rounds=10,
                             task_type="GPU")

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}


grid = {'learning_rate': [0.03],
        'depth': [4, 10],
        'l2_leaf_reg': [3, 9]}

grid_search_result = model_cb.grid_search(grid,
                                       cv=3,
                                       X=X_train_embeddings,
                                       y=y_train,
                                       plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Default metric period is 5 because MAE is/are not implemented for GPU
Default metric period is 5 because MAE is/are not implemented for GPU


: 

In [57]:
X_test_catboost = vec.transform(X_test)
 
y_pred = model_cb.predict(X_test_catboost)
y_pred = np.round(y_pred)

print(mean_absolute_error(y_test, y_pred))

CatBoostError: There is no trained model to use predict(). Use fit() to train model. Then use this method.

In [1]:
model_cb = CatBoostRegressor(learning_rate=0.005,
                            max_depth=10,
                            n_estimators=5,
                            loss_function='MAE',
                            thread_count=-1, 
                            random_seed=42, 
                            verbose=0, 
                            task_type = "GPU" 
                                
)


model_cb.fit(X_train_embeddings, y_train,  plot=True)

NameError: name 'CatBoostRegressor' is not defined

### Попробуем подбор гиперпараметов